In [ ]:
import boto3

client = boto3.client('elbv2')

In [ ]:
response = client.create_target_group(
    Name='geoff',
    HealthCheckEnabled=True,
    HealthCheckPath='/health',
    TargetType='lambda'
)

In [ ]:
print(response)

In [ ]:
cfclient = boto3.client('cloudformation')

In [ ]:
stack_id = 'arn:aws:cloudformation:us-east-1:427848627088:stack/mf3/46204ce0-3b82-11e9-9c8a-12967e4454e6'
logical_resid = 'MyCustomTargetGroup'

In [ ]:
response = cfclient.describe_stack_resources(
    StackName=stack_id,
)

In [ ]:
print(response)

In [ ]:
response = client.describe_target_groups(

    Names=[
        'mf3-hwFunction',
    ]
)

print(response['TargetGroups'][0]['TargetGroupArn'])

In [ ]:
targetGroupArn = response['TargetGroups'][0]['TargetGroupArn']

In [ ]:
response = client.describe_load_balancers(
    Names=['geoffry']
)

In [ ]:
print(response['LoadBalancers'][0])

In [ ]:
listenerResponse = client.describe_listeners(
    LoadBalancerArn=response['LoadBalancers'][0]['LoadBalancerArn']
)

In [ ]:
print(listenerResponse)

In [ ]:
listener = [d for d in listenerResponse['Listeners'] if d['Protocol'] == 'HTTPS' and d['Port'] == 443][0]
print(listener)
listenerArn = listener['ListenerArn']
print(listenerArn)

In [ ]:
rulesResp = client.describe_rules(
    ListenerArn=listenerArn
)

In [ ]:
print(rulesResp)

In [ ]:
nonDefs = [r for r in rulesResp['Rules'] if r['Priority'] != 'default']
print(nonDefs)

In [ ]:
priorities = [r['Priority'] for r in rulesResp['Rules'] if r['Priority'] != 'default']
print(list(map(int, priorities)))
print(max(list(map(int, priorities))))

In [ ]:
def next_order_no(listenerArn, elbClient):
    rulesResp = elbClient.describe_rules(
        ListenerArn=listenerArn
    )
    
    priorities = [r['Priority'] for r in rulesResp['Rules'] if r['Priority'] != 'default']
    if not priorities:
        return 1
    else:
        return max(list(map(int, priorities))) + 1

In [ ]:
next_order_no(listenerArn, client)

In [ ]:
def has_match_all_condition(rule):
    if not rule['Conditions'] or len(rule['Conditions']) == 0:
        return False
    
    for cond in rule['Conditions']:
        if cond['Field'] and cond['Field'] == 'path-pattern':
            values = cond['Values']
            
            for v in values:
                if v == '*' or v == '/*':
                    return True

    return False
    

In [ ]:
def star_path_rule(listenerArn, elbClient):
    rulesResp = elbClient.describe_rules(
        ListenerArn=listenerArn
    )
    
    
    rules = rulesResp['Rules']
    
    for r in rules:
        if has_match_all_condition(r):
            return r
    
    return None

In [ ]:
starPathRule = star_path_rule(listenerArn, client)
print(starPathRule)

In [ ]:
def change_priority(r, newPriority, elbClient):
    response = elbClient.set_rule_priorities(
        RulePriorities=[
            {
                'RuleArn': r['RuleArn'],
                'Priority': newPriority
            },
        ]
    )
    
    print(response)

In [ ]:
change_priority(starPathRule, 50000, client)

In [ ]:
def get_listener_arn(lb_name, proto, port, elbClient):
    # First get the load balancer arn
    response = elbClient.describe_load_balancers(
        Names=[lb_name]
    )
    
    lbArn = response['LoadBalancers'][0]['LoadBalancerArn']
    
    # Now get the listeners and filter by desired protocol and port
    listenerResponse = elbClient.describe_listeners(
        LoadBalancerArn=lbArn
    )
    
    listener = [d for d in listenerResponse['Listeners'] if d['Protocol'] == proto and d['Port'] == port][0]
    
    return listener['ListenerArn']
    
    
    
    

In [ ]:
get_listener_arn('geoffry', 'HTTPS',443, client)

In [ ]:
next = next_order_no(get_listener_arn('geoffry', 'HTTPS',443, client),client)
print(next)

In [ ]:
print(listenerArn)
print(next)
print(targetGroupArn)

In [ ]:
response = client.create_rule(
        ListenerArn=listenerArn,
        Conditions=[
            {
                'Field': 'path-pattern',
                'Values': [
                    '/foooo',
                ]
            },
        ],
        Priority=next,
        Actions=[
            {
                'Type': 'forward',
                'TargetGroupArn': targetGroupArn,

                'Order': 1,
            },
        ]
)

In [ ]:
print(response)

In [ ]:
rulesResp = client.describe_rules(
    ListenerArn=listenerArn
)
print(rulesResp)

In [ ]:
theRule = [r for r in rulesResp['Rules'] if r['Priority'] != 'default' and len(r['Actions']) == 1 and r['Actions'][0]['TargetGroupArn'] == targetGroupArn][0]
print(theRule)

In [ ]:
response = client.delete_rule(
    RuleArn=theRule['RuleArn']
)

print(response)